<a href="https://colab.research.google.com/github/abdulrahmanalaa123/Graduation_Project/blob/Honda/tsfel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pickle5
!pip install tsfel
import pickle5 as pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn 
import pandas as pd
import json
import tensorflow as tf
import tsfel as tsfel
from re import L
from operator import itemgetter
from google.colab import files
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import models   # for model loading
from sklearn.model_selection import GridSearchCV    
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from statsmodels.tsa.stattools import adfuller

#%pip install pandas==1.4.1       unpickle problem
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

with open("/content/drive/MyDrive/Colab Notebooks/hr(tuples).pkl", "rb") as f: 
    data = pickle.load(f)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256434 sha256=b9d6fd460940b0f809dc7ce3f58d18b9c81feb9ae1d73097df83e202b732b503
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00
Mounted at /content/drive


1_replace ur data here

In [ ]:
def evensampler(data):
    mask = data["Stage"] == "STIMULUS"
    participants = data["Participant"].unique()
    trainval = int(0.9*len(participants))
    trainpar = participants[:trainval]
    testpar = participants[trainval:]
    masktrain = data["Participant"].isin(trainpar)
    masktest = data["Participant"].isin(testpar)
    train = data.loc[mask&masktrain,data.columns[2:-13]].values
    traintarget = data.loc[mask&masktrain,"Target"].values
    test = data.loc[mask&masktest,data.columns[2:-13]].values
    testtarget = data.loc[mask&masktest,"Target"].values

    return train,test,traintarget,testtarget
def getter(x):
    if isinstance(x,tuple):
      
        return itemgetter(0)(x)
    else:
        return x
def splittuple(dat):
  return dat.applymap(lambda x:getter(x))

hrdata=splittuple(data)
xtrain1,xtest1,ytrain1,ytest1 = evensampler(hrdata)
print(xtrain1.T)
cols=hrdata.columns[2:-13]
X_train_sig = pd.DataFrame(xtrain1.T, index=[cols])
Tx_train=X_train_sig
#Tx_train = Tx_train.replace(0, np.nan)
print(hrdata)

[[87 81 85 ... 77 76 84]
 [87 81 85 ... 77 76 84]
 [87 81 85 ... 76 76 84]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
      Participant           Stage  0:0.0  0:0.099791  0:0.199582  0:0.299373  \
0              22        STIMULUS     87          87          87          87   
1              22  QUESTIONNAIRES     85          85          85          85   
2              22         WASHOUT     80          80          80          80   
3              22        STIMULUS     81          81          81          81   
4              22  QUESTIONNAIRES     83          82          83          83   
...           ...             ...    ...         ...         ...         ...   
1339           64        STIMULUS     78          78          78          78   
1340           64  QUESTIONNAIRES     77          77          77          77   
1341           64         WASHOUT      0           0           0           0   
1342           64        STIMULUS     69    

In [ ]:
xtrain1.shape

(407, 6290)

2_run this to extract features

In [ ]:
def packer(data: pd.Series,window_size:int,feature_type=None):
    print("data shape before extracting features",data.shape)
    cfg_file = tsfel.get_features_by_domain(feature_type)   # If no argument is passed retrieves all available features, else statistical,temporal,spectral,wavelet,information
    X_train = tsfel.time_series_features_extractor(cfg_file,data.T, fs=1, window_size=window_size)    # Receives a time series sampled at 50 Hz, divides into windows of size 250 (i.e. 5 seconds) and extracts all features
    print("features shape is :",X_train.shape)
    features = [tuple(row) for row in X_train.values]
    print("number of windows :",len(features))
    print("number of features",X_train.shape[1])
    return features

def featuremydata(data,cols,window,feature=None):
  #packed = pd.DataFrame(columns = column)
  X_train_sig = pd.DataFrame(data.T, index=[cols])
  packed = X_train_sig.T.apply(packer,axis =1,feature_type=feature,window_size=window)
  #print("packed is ",packed)
  #print(packed.shape)                                                                                                                 #getting shape of data 
  sub = [[elem for elem in packed[index]] for index in packed.index[:]]
  #print("sub  is :",sub)                                                                                                                 #features of participant n,in the x window
  #print(len(sub))                                                                                                                          #consider it the number of participants or given signals
  print("columns number",data.shape[1]/window)#window numbers
  column = np.arange(int(data.shape[1]/window))
  final = pd.DataFrame(sub,columns = column)
  print(final)
  return final
def packer_cols(data: pd.Series,window_size:int,feature_type=None):
    print(data.shape)
    cfg_file = tsfel.get_features_by_domain(feature_type)                                                        # If no argument is passed retrieves all available features
    X_train = tsfel.time_series_features_extractor(cfg_file,data.T, fs=1, window_size=window_size)    # Receives a time series sampled at 50 Hz, divides into windows of size 250 (i.e. 5 seconds) and extracts all features
    print(X_train.shape)
    return X_train.columns

def trial (train,test,window_size):
  f_cols=packer_cols(hrdata.iloc[1,2:-13],window_size)
  print("features names is :",f_cols)
  cols=hrdata.columns[2:-13]
  test=featuremydata(xtest1,cols,window_size)
  train=featuremydata(xtrain1,cols,window_size)
  return train,test,f_cols

temporaltrain,temporaltest,f_cols=trial(xtrain1,xtest1,1000)
#train.to_pickle("/content/drive/MyDrive/Colab Notebooks/hr(tuples)tsfel_temp'train'.pkl")
#test.to_pickle("/content/drive/MyDrive/Colab Notebooks/hr(tuples)tsfel_temp'test'.pkl")
#test.to_pickle("/content/drive/MyDrive/Colab Notebooks/hr(tuples)packked'test'.pkl")

(6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
(6, 389)
features names is : Index(['0_Absolute energy', '0_Area under the curve', '0_Autocorrelation',
       '0_Centroid', '0_ECDF Percentile Count_0', '0_ECDF Percentile Count_1',
       '0_ECDF Percentile_0', '0_ECDF Percentile_1', '0_ECDF_0', '0_ECDF_1',
       ...
       '0_Wavelet variance_0', '0_Wavelet variance_1', '0_Wavelet variance_2',
       '0_Wavelet variance_3', '0_Wavelet variance_4', '0_Wavelet variance_5',
       '0_Wavelet variance_6', '0_Wavelet variance_7', '0_Wavelet variance_8',
       '0_Zero crossing rate'],
      dtype='object', length=389)
data shape before extracting features (6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
data shape before extracting features (6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
data shape before extracting features (6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
data shape before extracting features (6290,)
*** Feature extraction started ***


...


*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
data shape before extracting features (6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
data shape before extracting features (6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
data shape before extracting features (6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
data shape before extracting features (6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
features shape is : (6, 389)
number of windows : 6
number of features 389
columns number 6.29
                                                     0  \
0    (6893388.0, 82793.0, 6893388.0, 485.2500533844...   
1    (6774542.0, 82140.5, 6774542.0, 484.5065791901...   
2    (6797216.0, 82302.0, 6797216.0, 485.1574847702...   
3    (4743162.0, 57185.5, 4743162.0, 329.3391123895...   
4    (2890036.0, 37992.5, 2890036.0, 250.7773833959...   
..                                                 ...   
402  (4880557.0, 58018.0, 4880557.0, 351.4311149321...   
403  (3185183.0, 39472.5, 3185183.0, 238.1421833533...   
404  (6068209.0, 77722.5, 6068209.0, 486.4282983661...   
405  (6310609.0, 79140.0, 6310609.0, 522.2535300475...   
406  (6246093.0, 78799.0, 6246093.0, 495.9825500516...   

                                                     1  \
0    (6701879.0, 81679.0, 6701879.0, 525.1561958967...   
1    (1424560.0, 17741.0, 1424560.0, 112.6529433649...  

In [ ]:
f_cols=packer_cols(hrdata.iloc[1,2:-13],1000)

(6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
(6, 389)


3_run this to change shape

In [ ]:
def feature_data_shaper(data,target):
############################## convert to 3d n_participants*number of windows*number of features
  data_array = data.values
  print("3d data but with tuples in cell",data_array.shape)
  max_metrics = max([len(t) for t in data.iloc[:, 0]])
  padded_data_array = np.zeros((data.shape[0],data.shape[1], max_metrics))
  for i in range(data.shape[0]):
      for j in range(data.shape[1]):
          padded_data_array[i, j, :len(data_array[i, j])] = data_array[i, j]

  # Print the shape of the 3D array
  print("3d data",padded_data_array.shape)
  ######################### repeating the target variables "n_participants * number of windows" times
  target=np.repeat(target,padded_data_array.shape[1])

  ######################
  # Flatten the 3D numpy array into a 2D numpy array
  n_windows = padded_data_array.shape[0] * padded_data_array.shape[1]
  data_flat = np.reshape(padded_data_array, (n_windows, -1))
  print("final data shape prepared for learning",data_flat.shape,len(target))
  # Concatenate the flattened feature data with the target labels
  data_with_target = np.hstack((data_flat, target.reshape(-1, 1)))
  return data_with_target
#prepare data for random forrest
rftrain=feature_data_shaper(temporaltrain,ytrain1)
rftest=feature_data_shaper(temporaltest,ytest1)
#prepare column name for random forest
f_cols=np.array(f_cols)
f_cols=np.append(f_cols,"target")
f_cols = [s.replace('0_', '') for s in f_cols]
print(f_cols)

3d data but with tuples in cell (407, 1)
3d data (407, 1, 389)
final data shape prepared for learning (407, 389) 407
3d data but with tuples in cell (55, 1)
3d data (55, 1, 389)
final data shape prepared for learning (55, 389) 55
['Absolute energy', 'Area under the curve', 'Autocorrelation', 'Centroid', 'ECDF Percentile Count_0', 'ECDF Percentile Count_1', 'ECDF Percentile_0', 'ECDF Percentile_1', 'ECDF_0', 'ECDF_1', 'ECDF_2', 'ECDF_3', 'ECDF_4', 'ECDF_5', 'ECDF_6', 'ECDF_7', 'ECDF_8', 'ECDF_9', 'Entropy', 'FFT mean coefficient_0', 'FFT mean coefficient_1', 'FFT mean coefficient_10', 'FFT mean coefficient_100', 'FFT mean coefficient_101', 'FFT mean coefficient_102', 'FFT mean coefficient_103', 'FFT mean coefficient_104', 'FFT mean coefficient_105', 'FFT mean coefficient_106', 'FFT mean coefficient_107', 'FFT mean coefficient_108', 'FFT mean coefficient_109', 'FFT mean coefficient_11', 'FFT mean coefficient_110', 'FFT mean coefficient_111', 'FFT mean coefficient_112', 'FFT mean coeffici

In [ ]:
rftest.shape

(6875, 160)

4_run this for modeling and test

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define the feature columns
feature_columns = [tf.feature_column.numeric_column(key=name) for name in f_cols]

# Split the data into features and target variable
df=pd.DataFrame(rftrain[:, :-1])
df = df.fillna(0)
X=np.array(df)
y = rftrain[:, -1]
# Convert the target variable to integers
le = LabelEncoder()
y = le.fit_transform(y)
#y = y.astype(int)

rf = RandomForestClassifier(n_estimators=500)
rf.fit(X, y)

df=pd.DataFrame(rftest[:, :-1])
df = df.fillna(0)
X_test = np.array(df)
y_test = rftest[:, -1]
y_test = le.fit_transform(y_test)

# Use the trained model to predict the target values for the test data
y_pred = rf.predict(X_test)

# Compute the accuracy of the predictions

accuracy = accuracy_score(y_test, y_pred)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.6909090909090909


In [ ]:
f_cols=packer_cols(hrdata.iloc[1,2:-13],150)


(6290,)
*** Feature extraction started ***



*** Feature extraction finished ***
(41, 209)


In [ ]:
################################################################################################################################################################
with open("/content/drive/MyDrive/Colab Notebooks/hr(tuples)packked.pkl", "rb") as f: 
    dataa = pickle.load(f)
print(dataa)

                                                   0    \
0    (374996.0, 84.87, 374996.0, 0.4870059947306104...   
1    (337055.0, 80.46, 337055.0, 0.4954076931064662...   
2    (360598.0, 83.22, 360598.0, 0.4867850625904747...   
3    (357536.0, 82.87, 357536.0, 0.4867051150093977...   
4    (320707.0, 78.46000000000001, 320707.0, 0.4749...   
..                                                 ...   
402  (378476.0, 85.27, 378476.0, 0.4931280186854648...   
403  (330169.0, 79.63999999999999, 330169.0, 0.4908...   
404  (290177.0, 74.64999999999999, 290177.0, 0.4884...   
405  (284477.0, 73.92999999999999, 284477.0, 0.4819...   
406  (346812.0, 81.62, 346812.0, 0.4848352998166153...   

                                                   1    \
0    (386103.0, 86.1, 386103.0, 0.5004144489941803,...   
1    (342635.0, 81.12, 342635.0, 0.4887527543887811...   
2    (364531.0, 83.69, 364531.0, 0.4848340470357803...   
3    (366400.0, 83.88999999999999, 366400.0, 0.4949...   
4    (278169.